In [2]:
import pandas as pd
import timeit

In [3]:
def prepare():
    file = "household_power_consumption.txt"
    df = pd.read_csv(file, sep=";", header=None, low_memory=False)
    df.columns = ['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity',
                  'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
    df = df.drop(df.index[0])
    df = df[df.Global_active_power != '?']
    return df

In [36]:
# 1

SETUP_CODE = '''
import pandas as pd
from __main__ import prepare'''
TEST_CODE = '''
df = prepare()
print(df[df.Global_active_power.astype('float') > 5])'''
print(timeit.repeat(setup=SETUP_CODE, stmt=TEST_CODE, number=1))

               Date      Time Global_active_power Global_reactive_power  \
2        16/12/2006  17:25:00               5.360                 0.436   
3        16/12/2006  17:26:00               5.374                 0.498   
4        16/12/2006  17:27:00               5.388                 0.502   
12       16/12/2006  17:35:00               5.412                 0.470   
13       16/12/2006  17:36:00               5.224                 0.478   
...             ...       ...                 ...                   ...   
2069357  22/11/2010  18:40:00               5.408                 0.150   
2069358  22/11/2010  18:41:00               5.528                 0.144   
2071587  24/11/2010  07:50:00               5.172                 0.050   
2071588  24/11/2010  07:51:00               5.750                 0.000   
2072998  25/11/2010  07:21:00               5.074                 0.240   

         Voltage Global_intensity Sub_metering_1 Sub_metering_2 Sub_metering_3  
2        233.630  

In [38]:
# 2
SETUP_CODE = '''
import pandas as pd
from __main__ import prepare'''
TEST_CODE = '''
df = prepare()
print(df[df.Voltage.astype('float') > 235])'''

print(timeit.repeat(setup=SETUP_CODE, stmt=TEST_CODE, number=1))

               Date      Time Global_active_power Global_reactive_power  \
5        16/12/2006  17:28:00               3.666                 0.528   
6        16/12/2006  17:29:00               3.520                 0.522   
7        16/12/2006  17:30:00               3.702                 0.520   
8        16/12/2006  17:31:00               3.700                 0.520   
15       16/12/2006  17:38:00               4.054                 0.422   
...             ...       ...                 ...                   ...   
2075255  26/11/2010  20:58:00               0.946                 0.000   
2075256  26/11/2010  20:59:00               0.944                 0.000   
2075257  26/11/2010  21:00:00               0.938                 0.000   
2075258  26/11/2010  21:01:00               0.934                 0.000   
2075259  26/11/2010  21:02:00               0.932                 0.000   

         Voltage Global_intensity Sub_metering_1 Sub_metering_2 Sub_metering_3  
5        235.680  

KeyboardInterrupt: 

In [ ]:
# 3
SETUP_CODE = '''
import pandas as pd
from __main__ import prepare'''
TEST_CODE = '''
df = prepare()
print(df[(df.Global_intensity.astype('float') > 19) & (df.Global_intensity.astype('float') < 20) & df['Sub_metering_2'].astype('float') > df['Sub_metering_3'].astype('float')])'''
print(timeit.repeat(setup=SETUP_CODE, stmt=TEST_CODE, number=1))

In [9]:
#4
SETUP_CODE = '''
import pandas as pd
from __main__ import prepare'''
TEST_CODE = '''
df = prepare()
new_df = df.sample(n=500000, replace=False)
print(new_df.Sub_metering_1.astype('float').mean())'''
print(timeit.repeat(setup=SETUP_CODE, stmt=TEST_CODE, number=1))

1.115788


In [30]:
#5
SETUP_CODE = '''
import pandas as pd
from __main__ import prepare'''
TEST_CODE = '''
df = prepare()
times = df['Time'].str.replace(':', ' ').str.split()
df_filtered = df[(times.apply(lambda x: float(x[0]) >= 18)) & (df.Global_active_power.astype('float') > 6) & (df.Sub_metering_2.astype('float') > df.Sub_metering_1.astype('float')) & (df.Sub_metering_2.astype('float') > df.Sub_metering_3.astype('float'))]
first_half = df_filtered.iloc[:len(df_filtered)//2]
second_half = df_filtered.iloc[len(df_filtered)//2:]
selected_results = pd.concat([first_half.iloc[::3], second_half.iloc[::4]])
print(selected_results)'''

print(timeit.repeat(setup=SETUP_CODE, stmt=TEST_CODE, number=1))


               Date      Time Global_active_power Global_reactive_power  \
42       16/12/2006  18:05:00               6.052                 0.192   
45       16/12/2006  18:08:00               6.308                 0.116   
17495    28/12/2006  20:58:00               6.386                 0.374   
17499    28/12/2006  21:02:00               8.088                 0.262   
17502    28/12/2006  21:05:00               7.230                 0.152   
...             ...       ...                 ...                   ...   
2066467  20/11/2010  18:30:00               6.620                 0.314   
2066471  20/11/2010  18:34:00               6.266                 0.360   
2066475  20/11/2010  18:38:00               6.302                 0.376   
2066479  20/11/2010  18:42:00               6.238                 0.344   
2066483  20/11/2010  18:46:00               6.438                 0.308   

         Voltage Global_intensity Sub_metering_1 Sub_metering_2 Sub_metering_3  
42       232.930  